In [1]:
import pandas as pd
import numpy as np
import cellstates as cs
import matplotlib.pyplot as plt
import matplotlib

This notebook shows an example analysis of a dataset with cellstates. We are looking at 1937 pancreatic cells from one human donor. 

### Reference
Baron, M. et al. A Single-Cell Transcriptomic Map of the Human and Mouse Pancreas Reveals Inter- and Intra-cell Population Structure. Cell Syst. 3, 346-360.e4 (2016).

In [ ]:
filepath = '../test_data/baron/Gene_table.txt.gz'
df = pd.read_csv(filepath, sep='\t', index_col=0)
df.head()

In [ ]:
clusters = np.loadtxt('../test_data/baron/optimized_clusters.txt', dtype=np.int)
lmbd = np.loadtxt('../test_data/baron/dirichlet_pseudocounts.txt')
hierarchy_df = pd.read_csv('../test_data/baron/cluster_hierarchy.tsv', sep='\t')
score_df = pd.read_csv('../test_data/baron/hierarchy_gene_scores.tsv', sep='\t')
display(score_df.head())
display(hierarchy_df.head())

In [ ]:
data = df.values.astype(np.int)
data.shape

In [ ]:
clst = cs.Cluster(data, lmbd, clusters, max_clusters=max(clusters)+1)

# Visualization with PCA

In [ ]:
# calculate PCA
from sklearn.decomposition import PCA

n_scale = np.median(data.sum(axis=0))

tpm_data = n_scale*data/data.sum(axis=0)
X = np.log(tpm_data + 1).T
pca = PCA(n_components=16)
pca.fit(X)
X_pca = pca.transform(X)

pca_var = pca.explained_variance_ratio_

Plotting all cells and clusters on PCA quickly looks very messy

In [ ]:
# define colors
cluster_names, cluster_sizes = np.unique(clusters, return_counts=True)
colors = plt.cm.hsv(np.linspace(0, 1, clst.n_clusters))

fig, ax = plt.subplots(1, 1, figsize=(6,6))

# plot first 2 PCA components
c1, c2 = 0, 1
x = X_pca[:, c1]
y = X_pca[:, c2]
for c, color in zip(cluster_names, colors):
    cond = (clusters==c)
    label = f'{c}: n={np.sum(cond)}'
    ax.plot(x[cond], y[cond], '.', ms=3, label=label, color=color)
ax.set_xlabel(f'PC {c1+1}: {pca_var[c1]:.3f}')
ax.set_ylabel(f'PC {c2+1}: {pca_var[c2]:.3f}')
l = plt.legend(ncol=3)
l.set_bbox_to_anchor([1., 1])

fig.suptitle('PCA showing all cellstates')
    
plt.show()

One way we can make this plot tidier is by finding the  **modal gene expression state** of each cluster and only plotting these. 

*NB*: The inferred modal gene expression state of small clusters might be shifted w.r.t. the positions of the cells themselves.

In [ ]:
# find modal gene expression states of all cellstates
all_gene_expression_states = np.vstack([clst.get_expressionstate(c) for c in cluster_names])

log_tpm_states = np.log(all_gene_expression_states*n_scale+1)
X_pca_states = pca.transform(log_tpm_states)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8,))

c1, c2 = 0, 1

# plot centres of cellstates with cells in background
ax = axs[0]
ax.plot(X_pca[:, c1], X_pca[:, c2], '.', ms=1, color='gray')
ax.scatter(X_pca_states[:, c1], X_pca_states[:, c2], 
           s=cluster_sizes, 
           c=colors)
ax.set_xlabel(f'PC {c1+1}: {pca_var[c1]:.3f}')
ax.set_ylabel(f'PC {c2+1}: {pca_var[c2]:.3f}')

# plot previous PCA for comparison
ax = axs[1]
x = X_pca[:, c1]
y = X_pca[:, c2]
for c, color in zip(cluster_names, colors):
    cond = (clusters==c)
    label = f'{c}: n={np.sum(cond)}'
    ax.plot(x[cond], y[cond], '.', ms=3, label=label, color=color)
ax.set_xlabel(f'PC {c1+1}: {pca_var[c1]:.3f}')
ax.set_ylabel(f'PC {c2+1}: {pca_var[c2]:.3f}')
l = plt.legend(ncol=3)
l.set_bbox_to_anchor([1., 1])


fig.suptitle(f'PCA showing average gene expression states', size=20)
plt.show()

Another way to better understand the large-scale structure and get a tidier representation is to look at **merged clusters**:

In [ ]:

nc = 12 # number of clusters
merged_clusters = cs.clusters_from_hierarchy(hierarchy_df, cluster_init=clusters, steps= - nc + 1)

cluster_names, cluster_sizes = np.unique(merged_clusters, return_counts=True)
colors = plt.cm.hsv(np.linspace(0, 1, nc+1))

fig, ax = plt.subplots(1, 1, figsize=(6,6))

# plot first 2 PCA components
c1, c2 = 0, 1
x = X_pca[:, c1]
y = X_pca[:, c2]
for c, color in zip(cluster_names, colors):
    cond = (merged_clusters==c)
    label = f'{c}: n={np.sum(cond)}'
    ax.plot(x[cond], y[cond], '.', ms=3, label=label, color=color)
ax.set_xlabel(f'PC {c1+1}: {pca_var[c1]:.3f}')
ax.set_ylabel(f'PC {c2+1}: {pca_var[c2]:.3f}')
l = plt.legend(ncol=1)
l.set_bbox_to_anchor([1., 1])

fig.suptitle(f'PCA showing cellstates merged to {nc} clusters')
plt.show()

# Analysis of cluster hierarchy

The default scipy visualization includes all cellstates as leaves and gives a general overview of the data structure. 

*NB*: single leaves cannot be colored in scipy and are shown in gray, not the color assigned above

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 6))
R = cs.plot_hierarchy_scipy(hierarchy_df, n_groups=nc, colors=colors)

# setting the yscale to symlog allows us to better see the fine structure of the tree
ax.set_yscale('symlog', linthresh=1e2)

In [ ]:
# if you like working with the scipy hierarchy module, you can use
Z, labels = cs.get_scipy_hierarchy(hierarchy_df, return_labels=True)

The default ete3 visualization includes all cells as leaves (i.e. horizontal width of clusters correspond to their sizes) and gives a general overview of the data structure. 

*NB*: The leaf order may be changed compared to the scipy plot

In [ ]:
# I need to properly set the DISPLAY environment to run ete3 on a remote server
# You should probably not run this
import os
os.environ['DISPLAY'] = 'localhost:11.0'  # match to 
!echo $DISPLAY

In [ ]:
# this built-in function automatically generates a tree visualization with ete3
# example plot with 2 higher-order groups and cellstates as leaves (size proportional to number of cells)
t, ts = cs.plot_hierarchy_ete3(hierarchy_df, clst.clusters, n_groups=nc, 
                               colors=colors, show_cells=False)
ts.show_leaf_name = True  # add leaf labels
t.render('%%inline', tree_style=ts)

In [ ]:
# example plot with 5 higher-order groups and cells as leaves (horizontal distance proportional to number of cells)
t, ts = cs.plot_hierarchy_ete3(hierarchy_df, clst.clusters, n_groups=nc, 
                               colors=colors, show_cells=True)
ts.scale=3e-4  # change vertical scale of figure
t.render('%%inline', tree_style=ts)

In [ ]:
# If you like a different tree plotting program that uses newick format inputs, you can use
# check the docstring for further options
newick_string = cs.hierarchy_to_newick(hierarchy_df, clst.clusters, cell_leaves=False)

from ete3 import Tree
# generate ete3 Tree
t = Tree(newick_string, format=1)

Each of the merges in the tree is associated with genes that are differentially expressed between the two branches. For example, for the last merge we find the following top genes:

In [ ]:
split_id = 0  # first split in tree

gene_scores = score_df.iloc[-split_id - 1, 4:]
gene_scores.sort_values()

The list shows that *INS* is expressed very differently in both branches while any variation in *SST* expression is very consistent with being noise.

In the original paper, *INS* is marked as a marker gene for Beta cells.

Below, we show *INS* expression of cellstates in the PCA plot

In [ ]:
my_gene = 'INS'
gene_id = np.argwhere(df.index.values==my_gene)[0,0]
expression = log_tpm_states[:, gene_id]
norm = plt.Normalize(vmin=expression.min(), vmax=expression.max())
cmap = plt.cm.RdBu_r

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,6))

c1, c2 = 0, 1

# plot centres of cellstates
ax.plot(X_pca[:, c1], X_pca[:, c2], '.', ms=1, color='gray')
sc = ax.scatter(X_pca_states[:, c1], X_pca_states[:, c2], 
                s=clst.cluster_sizes, 
                c=expression,
                cmap='RdBu_r')
ax.set_xlabel(f'PC {c1+1}: {pca_var[2*c1+1]:.3f}')
ax.set_ylabel(f'PC {c2+1}: {pca_var[2*c2+1]:.3f}')
plt.suptitle(my_gene, size=20)
plt.colorbar(sc)
plt.show()

It would be interesting to see how this gene is expressed across the tree. For that purpose, we use the ete3 tree plotting library and create a custom function. 

In [ ]:
from ete3 import TreeStyle, NodeStyle


def get_TreeStyle_expression(t, gene_name, log_tpm_states, hierarchy_df, clst,
                             cmap='RdBu_r', linewidth = 2, leaf_scale=0.2):
    
    gene_id = np.flatnonzero(df.index.values==gene_name)[0]
    
    ts = TreeStyle()
    ts.show_leaf_name=True
    ts.scale = 3e-5
    ts.rotation = 90
    
    expression = log_tpm_states[:, gene_id]
    norm = plt.Normalize(vmin=expression.min(), vmax=expression.max())
    cmap = getattr(plt.cm, cmap)
    
    cluster_map = dict(zip(np.flatnonzero(clst.cluster_sizes), range(clst.n_clusters)))
    
    # set up colouring of leaves/cellstates which are called C0, C1, ...
    for leaf in t.iter_leaves():
        cluster_id = cluster_map[int(leaf.name[1:])]
        # find expression of that gene
        E = expression[cluster_id]
        color = matplotlib.colors.to_hex( cmap(norm(E)) )

        style = NodeStyle()
        # make size proportional to size of cluster
        style['size'] = clst.cluster_sizes[cluster_id]*leaf_scale

        # make color 
        style['fgcolor'] = color
        style["vt_line_color"] = color
        style["hz_line_color"] = color
        style['vt_line_width'] = linewidth
        style['hz_line_width'] = linewidth

        leaf.set_style(style)

    unmerged_clusters = clst.clusters.copy()
    
    # Iterate through internal nodes of tree
    n_steps = len(hierarchy_df)
    for i, row in hierarchy_df.iterrows():
        c_new = row.cluster_new
        c_old = row.cluster_old
        # merge clusters
        clst.combine_two_clusters(c_new, c_old)

        # find expression of gene in new merged cluster
        E = np.log(clst.get_expressionstate(c_new)[gene_id]*n_scale + 1)
        color = matplotlib.colors.to_hex( cmap(norm(E)) )

        node_name = f'I{n_steps-i-1}'
        nodes = t.search_nodes(name=node_name)
        if nodes:
            node = nodes[0]
            style = NodeStyle()
            style["vt_line_color"] = color
            style["hz_line_color"] = color
            style['vt_line_width'] = linewidth
            style['hz_line_width'] = linewidth
            style['size'] = 0
            node.set_style(style)
    
    clst.set_clusters(unmerged_clusters)
    return ts

In [ ]:
# Beta (INS)
ts = get_TreeStyle_expression(t, my_gene, log_tpm_states, hierarchy_df, clst)
t.render('%%inline', tree_style=ts)

Let's try a few different genes - all are marker genes from the original paper. This allows us to see which branches of the tree correspond to certain cell types.

In [ ]:
# Alpha (GCG)
ts = get_TreeStyle_expression(t, 'GCG', log_tpm_states, hierarchy_df, clst)
t.render('%%inline', tree_style=ts)

In [ ]:
# Delta (SST)
ts = get_TreeStyle_expression(t, 'SST', log_tpm_states, hierarchy_df, clst)
t.render('%%inline', tree_style=ts)

In [ ]:
# Gamma (PPY)
ts = get_TreeStyle_expression(t, 'PPY', log_tpm_states, hierarchy_df, clst)
t.render('%%inline', tree_style=ts)

In [ ]:
# Epsilon (GHRL)
ts = get_TreeStyle_expression(t, 'GHRL', log_tpm_states, hierarchy_df, clst)
t.render('%%inline', tree_style=ts)

leaves are labeled C*n* where *n* is the cellstate ID, internal nodes are labeled I0 (root) ... I*(x-1)* where *x* is the number of splits in the tree. The last line in `hierarchy_df` and `gene_scores` correspond to I0, the second last to I1, etc. 

We can also easily look at parts of the tree - for example to examine the transcriptional diversity in *INS* expressing beta cells. The first split therein divides cluster labels 13 and 5. 

In [ ]:
# this finds the subbranch of the tree containing clusters 13 and 5
branch = t.get_common_ancestor(['C13', 'C5']) 
branch.name

In [ ]:
n_steps = len(hierarchy_df)

split_id = 7  # id of internal node (branch name)
merge_id = n_steps - split_id - 1  # corresponding line in hierarchy_df
merge_id

In [ ]:
hierarchy_df.tail(10)

In [ ]:
# first check out interesting genes corresponding to this split
gene_scores = score_df.iloc[-split_id-1, 4:]
gene_scores.sort_values()

In [ ]:
# plotting this subbranch works the same as before
ts = get_TreeStyle_expression(branch, 'IAPP', log_tpm_states, hierarchy_df, clst)
ts.scale=1e-4  # adjust the scale to see more details at the bottom
branch.render('%%inline', tree_style=ts)

In [ ]:
# zoom into the left part of the tree in the same way as previously
branch = t.get_common_ancestor(['C38', 'C1'])
print(branch.name)
ts = get_TreeStyle_expression(branch, 'IAPP', log_tpm_states, hierarchy_df, clst)
ts.scale = 1e-3
branch.render('%%inline', tree_style=ts)